## Isolation Forest HBK

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import pickle
import warnings
warnings.filterwarnings('ignore')
#Mostrar todas las columnas de un dataframe
pd.set_option('display.max_columns', None)

In [ ]:
#Seleccionar ruta de la data
dataframe = pd.read_csv('../data/modeling/02_preprocessed.csv',sep= "|", encoding='latin')
dataframe.shape

In [ ]:
#Validador
c = dataframe['PROFECIIU'].value_counts(sort=False,dropna=False)
p = dataframe['PROFECIIU'].value_counts(sort=False,dropna=False, normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

In [ ]:
dataset=dataframe.loc[((dataframe['MTODOLARIZADO']>10000) & (dataframe['MTODOLARIZADO']<200000)) & ((dataframe['PERIODO']>202102) & (dataframe['PERIODO']<202108))]
dataset.shape

In [ ]:
ds=dataset[['MTODOLARIZADO','PAISRIESGO_MONTO','TIPPER','FLGARCHIVONEGATIVO','CTDEVAL','CTD_BENEFICIARIO','INDICE_CONCENTRADOR','RATIO']]

In [ ]:
#Modelo base IF
clf_base = IsolationForest(n_estimators = 500, contamination=0.001, random_state=7)
clf_base.fit(ds)

In [ ]:
#score de silueta
datasetv1 = dataset.copy()
datasetv1['IF_LABEL']=clf_base.predict(ds)

In [ ]:
datasetv1.groupby(['PERIODO','IF_LABEL']).size()

### Test

In [ ]:
df_test=dataframe.loc[((dataframe['MTODOLARIZADO']>10000) & (dataframe['MTODOLARIZADO']<200000)) & (dataframe['PERIODO']==202108)]
df_test.shape

In [ ]:
ds_test=df_test[['MTODOLARIZADO','PAISRIESGO_MONTO','TIPPER','FLGARCHIVONEGATIVO','CTDEVAL','CTD_BENEFICIARIO','INDICE_CONCENTRADOR', 'RATIO']]

In [ ]:
#Aplico el modelo cargado sobre la data std
df_test['Outlier']=clf_base.predict(ds_test)
df_test.head()

In [ ]:
#Guardar el modelo
pickle.dump(clf_base, open('../src/02_models/if_model_39.model', 'wb'))

In [ ]:
#Tamaño clusters
df_test.groupby('Outlier').Outlier.count()

In [ ]:
df_test.to_csv('../data/modeling/04_predicted.csv',index=False,sep='|',header=True)